In [2]:
import numpy as np
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb
import matplotlib.pyplot as plt

In [3]:
radius = 3
n_points = 8 * radius

In [4]:
def overlay_labels(image, lbp, labels):
    mask = np.logical_or.reduce([lbp == each for each in labels])
    return label2rgb(mask, image=image, bg_label=0, alpha=0.5)


def highlight_bars(bars, indexes):
    for i in indexes:
        bars[i].set_facecolor('r')

In [14]:
image = data.coins()
lbp = local_binary_pattern(image, n_points, radius, 'ror')

In [6]:
def hist(ax, lbp):
    n_bins = int(lbp.max() + 1)
    return ax.hist(lbp.ravel(), density=True, bins=n_bins, range=(0, n_bins),
                   facecolor='0.5')

In [ ]:
# plot histograms of LBP of textures
fig, (ax_img, ax_hist) = plt.subplots(nrows=2, ncols=3, figsize=(9, 6))
plt.gray()

titles = ('edge', 'flat', 'corner')
w = width = radius - 1
edge_labels = range(n_points // 2 - w, n_points // 2 + w + 1)
flat_labels = list(range(0, w + 1)) + list(range(n_points - w, n_points + 2))
i_14 = n_points // 4            # 1/4th of the histogram
i_34 = 3 * (n_points // 4)      # 3/4th of the histogram
corner_labels = (list(range(i_14 - w, i_14 + w + 1)) +
                 list(range(i_34 - w, i_34 + w + 1)))

label_sets = (edge_labels, flat_labels, corner_labels)

for ax, labels in zip(ax_img, label_sets):
    ax.imshow(overlay_labels(image, lbp, labels))

for ax, labels, name in zip(ax_hist, label_sets, titles):
    counts, _, bars = hist(ax, lbp)
    highlight_bars(bars, labels)
    ax.set_ylim(top=np.max(counts[:-1]))
    ax.set_xlim(right=n_points + 2)
    ax.set_title(name)

ax_hist[0].set_ylabel('Percentage')
for ax in ax_img:
    ax.axis('off')

In [6]:
import os
import cv2
import dlib
import glob
import numpy as np
import pandas as pd
from skimage.transform import integral_image
from skimage.feature import local_binary_pattern
from skimage.feature import multiblock_lbp
from deepgaze.color_detection import BackProjectionColorDetector
from deepgaze.color_detection import RangeColorDetector

In [7]:
predictor_path = "../model/shape_predictor_68_face_landmarks.dat"
faces_folder_path = "/data/face/Data_Collection/"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [3]:
def get_one_face(img):
    face = None
    face_dets = detector(img, 1)
    print("Number of faces detected: {}".format(len(face_dets)))
    if len(face_dets) > 0:
        face = face_dets[0]
    return face

In [1]:
def lbph_extract(img, d=None, eps=1e-7):
    if d is None:
        d = get_one_face(img)
    no_points = 24
    radius = 8
    d_top = d.top()
    d_bottom = d.bottom()
    d_left = d.left()
    d_right = d.right()
    cropped = img[d_top:d_bottom, d_left:d_right]
    im_gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(im_gray, no_points, radius, method='uniform')
    (histogram, _) = np.histogram(lbp.ravel(),
                                  bins=np.arange(0, no_points + 3),
                                  range=(0, no_points + 2))
    # now we need to normalise the histogram so that the total sum=1
    histogram = histogram.astype("float")
    histogram /= (histogram.sum() + eps)
    return histogram

In [9]:
img = dlib.load_rgb_image('../img/t1.jpg')

In [11]:
hists = lbph_extract(img)

Number of faces detected: 1


In [16]:
hists

array([0.01473077, 0.01317401, 0.00568007, 0.00536217, 0.00640936,
       0.00841959, 0.00847569, 0.01100951, 0.01521229, 0.02216862,
       0.03400559, 0.05023235, 0.06129328, 0.05616018, 0.03723131,
       0.02513721, 0.01629688, 0.01229512, 0.00934523, 0.00815311,
       0.00627378, 0.00581564, 0.00717605, 0.0166849 , 0.01747029,
       0.52578703])

In [13]:
img1 = dlib.load_rgb_image('../img/screenshots/screenshot_0_2019-09-03-16-44-28.jpg')

In [14]:
hists1 = lbph_extract(img1)

Number of faces detected: 1


In [17]:
hists1

array([0.03196901, 0.02442479, 0.01566655, 0.01222685, 0.01118626,
       0.01352758, 0.01292057, 0.01610013, 0.0165337 , 0.01754538,
       0.02402012, 0.03497514, 0.04575673, 0.03751879, 0.02323968,
       0.01685166, 0.01057926, 0.01020349, 0.00927853, 0.01297838,
       0.01297838, 0.01789224, 0.01676494, 0.01849925, 0.01757429,
       0.5187883 ])